In [1]:
# Add names to pairs!

In [2]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from TransformerGlow import AdamWeightDecayOptimizer, FactorTrainer
import numpy as np
import torch
import random
import pandas as pd
import pickle
# import os, sys
# import time
# import datetime
# import torch.nn.functional as F
# from torch import nn



# random.seed(42)
torch.manual_seed(42) 
if torch.cuda.is_available():
  device = torch.device("cuda")
  print('There are %d GPU(s) available.' % torch.cuda.device_count())
  print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
  print('No GPU available, using the CPU instead.')
  device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: NVIDIA A40


In [3]:
def get_examples1(embeddings, gen_labels, attribute_ids, rand_seed=2, train=True):
    random.seed(rand_seed)
    female_embeddings = []
    male_embeddings = []
    female_gen_labels = []
    male_gen_labels = []
    female_attribute_ids = []
    male_attribute_ids = []
    female_logits = []
    male_logits = []

    # print(len(gen_labels))
    for key,i in enumerate(gen_labels):
        # print(i)
        if i==0:
            if logits[key]<-12:
                female_embeddings.append(embeddings[key])
                female_gen_labels.append(i)
                female_attribute_ids.append(attribute_ids[key])
                # female_logits.append(logits[key])
                # print(key)
        else:
            if logits[key]>12:
                male_embeddings.append(embeddings[key])
                # print(embeddings[key])
                male_gen_labels.append(i)
                male_attribute_ids.append(attribute_ids[key])
                # male_logits.append(logits[key])

    female_embeddings = torch.tensor(np.array(female_embeddings))
    female_attribute_ids = torch.tensor(np.array(female_attribute_ids))
    female_gen_labels = torch.tensor(np.array(female_gen_labels))
    # female_logits = torch.tensor(np.array(female_logits))
    
    male_embeddings = torch.tensor(np.array(male_embeddings))
    male_attribute_ids = torch.tensor(np.array(male_attribute_ids))
    male_gen_labels = torch.tensor(np.array(male_gen_labels))
    # male_logits = torch.tensor(np.array(male_logits))
    
    return female_embeddings, female_attribute_ids, female_gen_labels, male_embeddings, male_attribute_ids, male_gen_labels

In [4]:
def get_examples(embeddings, gen_labels, attribute_ids, logits, rand_seed, train=True):
    random.seed(rand_seed)
    female_embeddings = []
    male_embeddings = []
    female_gen_labels = []
    male_gen_labels = []
    female_attribute_ids = []
    male_attribute_ids = []
    female_logits = []
    male_logits = []

    # print(len(gen_labels))
    for key,i in enumerate(gen_labels):
        # print(i)
        if i==0:
            if logits[key]<-12:
                female_embeddings.append(embeddings[key])
                female_gen_labels.append(i)
                female_attribute_ids.append(attribute_ids[key])
                female_logits.append(logits[key])
                # print(key)
        else:
            if logits[key]>12:
                male_embeddings.append(embeddings[key])
                male_gen_labels.append(i)
                male_attribute_ids.append(attribute_ids[key])
                male_logits.append(logits[key])

    # female_embeddings_2 = female_embeddings.copy()
    # male_embeddings_2 = male_embeddings.copy()
    # random.shuffle(female_embeddings_2)
    # random.shuffle(male_embeddings_2)

    # female_data_pairs = [[x, y] for x, y in zip(female_embeddings, female_embeddings_2)]
    # male_data_pairs = [[x, y] for x, y in zip(male_embeddings, male_embeddings_2)]

    # gender_data_pairs = female_data_pairs + male_data_pairs
    # random.shuffle(gender_data_pairs)

    
    

    # if train==True:
    #     gender_data_pairs = torch.tensor(np.array(gender_data_pairs))
    #     return gender_data_pairs

    # else:
    female_embeddings = torch.tensor(np.array(female_embeddings))
    female_attribute_ids = torch.tensor(np.array(female_attribute_ids))
    female_gen_labels = torch.tensor(np.array(female_gen_labels))
    female_logits = torch.tensor(np.array(female_logits))
    
    male_embeddings = torch.tensor(np.array(male_embeddings))
    male_attribute_ids = torch.tensor(np.array(male_attribute_ids))
    male_gen_labels = torch.tensor(np.array(male_gen_labels))
    male_logits = torch.tensor(np.array(male_logits))
    
    return female_embeddings, female_attribute_ids, female_gen_labels, female_logits, male_embeddings, male_attribute_ids, male_gen_labels, male_logits

In [5]:
# for i in female_logits:
#     print(i)

In [6]:
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/embeddings.pkl', 'rb') as f:
#         embeddings = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/gender.pkl', 'rb') as f:
#         gen_labels = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/token_ids.pkl', 'rb') as f:
#         attribute_ids = pickle.load(f)
# with open('../viz/2_attribute_embeddings_80000_150_tokens_balanced_2000/logits.pkl', 'rb') as f:
#         logits = pickle.load(f)

In [7]:
# import json

# with open('eval_gender_pairs.json') as f:
#     d = json.load(f)
# male_eval_words = [i[0] for i in d]
# female_eval_words = [i[1] for i in d]

In [8]:
# import random
# train_limit = int(0.99*len(embeddings))
# random.Random(4).shuffle(embeddings)
# random.Random(4).shuffle(gen_labels)
# random.Random(4).shuffle(attribute_ids)
# embeddings = embeddings[:train_limit]
# gen_labels = gen_labels[:train_limit]
# attribute_ids = attribute_ids[:train_limit]
# logits = logits[:train_limit]

# # test_embeddings = embeddings[train_limit:]
# # test_gen_labels = gen_labels[train_limit:]
# # test_attribute_ids = attribute_ids[train_limit:]
# # test_logits = logits[train_limit:]


In [9]:
attribute_pair = "african_european"
model_path = "bertflow_model_"+attribute_pair
bertflow = torch.load( model_path+'/bertflow_rho_0999_factors_6.pth')

#########TEST#################

In [10]:
from transformers import BertTokenizer, BertForMaskedLM
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
vocabs = tokenizer.get_vocab()
vv = dict((v,k) for k,v in vocabs.items())

embedding_decoder = BertForMaskedLM.from_pretrained("bert-base-uncased").cls.cuda()
def decode_emb(emb):
    embedding_decoder.eval()
    with torch.no_grad():
        logits_list = embedding_decoder(torch.tensor(emb))
        id_list = torch.argmax(logits_list, dim=1)
        return id_list
    

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# from torch.utils.data import TensorDataset

# with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/embeddings_'+str(2779)+'.pkl', 'rb') as f:
#     embeddings1 = pickle.load(f)
# with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/gender_'+str(2779)+'.pkl', 'rb') as f:
#         gen_labels1 = pickle.load(f)
# with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/token_ids_'+str(2779)+'.pkl', 'rb') as f:
#         attribute_ids1 = pickle.load(f)
# # with open('../1_viz/first_attribute_embeddings_'+attribute_pair+'/logits_'+str(2779)+'.pkl', 'rb') as f:
# #         logits1 = pickle.load(f)


# female_embeddings1, female_attribute_ids1, female_gen_labels1, male_embeddings1, male_attribute_ids1, male_gen_labels1 = get_examples1(embeddings1, gen_labels1, attribute_ids1, train=False)
# female_data_pairs_test1 = TensorDataset(female_embeddings1, female_attribute_ids1, female_gen_labels1)
# male_data_pairs_test1 = TensorDataset(male_embeddings1, male_attribute_ids1, male_gen_labels1)

# female_dataloader1 = DataLoader(
#             female_data_pairs_test1,  # The training samples.
#             sampler = None, #RandomSampler(female_data_pairs_test), # Select batches randomly
#             batch_size = 20 # Trains with this batch size.
#         )

# male_dataloader1 = DataLoader(
#             male_data_pairs_test1,  # The training samples.
#             sampler = None, #RandomSampler(male_data_pairs_test), # Select batches randomly
#             batch_size = 20 # Trains with this batch size.
#         )





# bertflow.eval()

# female_z_list = []
# male_z_list = []
# f_embs = []
# m_embs = []
# f_ids = []
# m_ids = []
# f_logits = []
# m_logits = []

# # For each batch of training data...
# bertflow.eval()
# with torch.no_grad():
#     for step, f_batch in enumerate(female_dataloader1):
#         # print(f_batch.shape)
#         embs = f_batch[0]
#         ids = f_batch[1]
#         # logit_v = f_batch[3]
#         # print(embs.shape)
#         female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#         female_z_list.append(female_z)
#         f_embs.append(embs)
#         f_ids.append(ids)
#         # f_logits.append(logit_v)
#     for step, m_batch in enumerate(male_dataloader1):
#         embs = m_batch[0]
#         ids = m_batch[1]
#         # logit_v = m_batch[3]
#         # gen = m_batch[2]
#         male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#         male_z_list.append(male_z)
#         m_embs.append(embs)
#         m_ids.append(ids)
#         # m_logits.append(logit_v)        


# sum_fem = 0
# count = 0
# for j in range(len(female_z_list)):
#     # print("############", j)
#     for k in range(len(female_z_list[j][0])):
#         sum_fem = sum_fem + female_z_list[j][0][k]
#         count+=1
# avg_sum_fem = sum_fem/count


In [12]:
# import copy
# #########generate embeddings in gaussian space#################
# def gen_flow_embs(female_dataloader, male_dataloader):
#     bertflow.eval()
    
#     female_z_list = []
#     male_z_list = []
#     f_embs = []
#     m_embs = []
#     f_ids = []
#     m_ids = []
#     f_logits = []
#     m_logits = []
    
#     # For each batch of training data...
#     bertflow.eval()
#     with torch.no_grad():
#         # for step, f_batch in enumerate(female_dataloader):
#         #     # print(f_batch.shape)
#         #     embs = f_batch[0]
#         #     ids = f_batch[1]
#         #     logit_v = f_batch[3]
#         #     # gen = f_batch[2]
#         #     # print(embs.shape)
#         #     female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#         #     female_z_list.append(female_z)
#         #     f_embs.append(embs)
#         #     f_ids.append(ids)
#         #     f_logits.append(logits)
#         for step, m_batch in enumerate(male_dataloader):
#             embs = m_batch[0]
#             ids = m_batch[1]
#             logit_v = m_batch[3]
#             male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
#             male_z_list.append(male_z)
#             m_embs.append(embs)
#             m_ids.append(ids)
#             m_logits.append(logit_v)

#     # new_female_z_list = copy.deepcopy(female_z_list)
#     new_male_z_list = copy.deepcopy(male_z_list)

#     gender_transfer_count = 0
#     semantic_transfer_count = 0
#     non_conversion_count = 0
#     original_old_word = []
#     old_word = []
#     old_id = []
#     new_word = []
#     new_id = []
#     gender = []
#     logit_vals = []

#     # sum_fem = 0
#     # count = 0
#     # for j in range(len(female_z_list)):
#     #     # print("############", j)
#     #     for k in range(len(female_z_list[j][0])):
#     #         sum_fem = sum_fem + female_z_list[j][0][k]
#     #         count+=1
#     # avg_sum_fem = sum_fem/count

#     for j in range(len(male_z_list)):        
#         for k in range(len(male_z_list[j][0])):
#         # part1 = torch.tensor([female_z[0][0][15].cpu().detach().numpy() for i in female_z[0][0]])
#             # new_male_z_list[j][0][0][k] = female_z_list[j][0][0][k]
#             new_male_z_list[j][0][k] = avg_sum_fem
        
            
#         eee = bertflow(new_male_z_list[j], reverse=True)
#         eee = torch.squeeze(eee)
#         ffff = decode_emb(eee)
#         ffff
#         for key,i in enumerate(decode_emb(m_embs[j][:,:].to(device))):
#             original_old_word.append(vv[m_ids[j][key].item()])
#             old_word.append(vv[i.item()])
#             old_id.append(i.item())
#             new_word.append(vv[ffff[key].item()])
#             new_id.append(ffff[key].item())
#             gender.append(1)
#             logit_vals.append(m_logits[j][key].item())
    
#     return original_old_word, old_word, old_id, new_word, new_id, gender, logit_vals

In [13]:
import copy
#########generate embeddings in gaussian space#################
def gen_flow_embs1(female_dataloader):
    bertflow.eval()
    
    female_z_list = []
    male_z_list = []
    f_embs = []
    m_embs = []
    f_ids = []
    m_ids = []
    f_logits = []
    m_logits = []
    
    # For each batch of training data...
    bertflow.eval()
    with torch.no_grad():
        for step, f_batch in enumerate(female_dataloader):
            # print(f_batch.shape)
            embs = f_batch[0]
            ids = f_batch[1]
            logit_v = f_batch[3]
            # gen = f_batch[2]
            # print(embs.shape)
            female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
            female_z_list.append(female_z)
            f_embs.append(embs)
            f_ids.append(ids)
            f_logits.append(logits)
        # for step, m_batch in enumerate(male_dataloader):
        #     embs = m_batch[0]
        #     ids = m_batch[1]
        #     logit_v = m_batch[3]
        #     male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
        #     male_z_list.append(male_z)
        #     m_embs.append(embs)
        #     m_ids.append(ids)
        #     m_logits.append(logit_v)

    # new_female_z_list = copy.deepcopy(female_z_list)
    # new_male_z_list = copy.deepcopy(male_z_list)

    # gender_transfer_count = 0
    # semantic_transfer_count = 0
    # non_conversion_count = 0
    # original_old_word = []
    # old_word = []
    # old_id = []
    # new_word = []
    # new_id = []
    # gender = []
    # logit_vals = []

    sum_fem = 0
    count = 0
    for j in range(len(female_z_list)):
        # print("############", j)
        for k in range(len(female_z_list[j][0])):
            sum_fem = sum_fem + female_z_list[j][0][k]
            count+=1
    avg_sum_fem = sum_fem/count
    return avg_sum_fem

In [14]:
import copy
#########generate embeddings in gaussian space#################
def gen_flow_embs(female_dataloader, male_dataloader, avg_sum_fem):
    bertflow.eval()
    
    female_z_list = []
    male_z_list = []
    f_embs = []
    m_embs = []
    f_ids = []
    m_ids = []
    f_logits = []
    m_logits = []
    
    # For each batch of training data...
    bertflow.eval()
    with torch.no_grad():
        for step, f_batch in enumerate(female_dataloader):
            # print(f_batch.shape)
            embs = f_batch[0]
            ids = f_batch[1]
            logit_v = f_batch[3]
            # gen = f_batch[2]
            # print(embs.shape)
            female_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
            female_z_list.append(female_z)
            f_embs.append(embs)
            f_ids.append(ids)
            f_logits.append(logits)
        for step, m_batch in enumerate(male_dataloader):
            embs = m_batch[0]
            ids = m_batch[1]
            logit_v = m_batch[3]
            male_z = bertflow(embs.to(device), return_loss=False)  # Here z is the sentence embedding
            male_z_list.append(male_z)
            m_embs.append(embs)
            m_ids.append(ids)
            m_logits.append(logit_v)

    # new_female_z_list = copy.deepcopy(female_z_list)
    new_male_z_list = copy.deepcopy(male_z_list)

    gender_transfer_count = 0
    semantic_transfer_count = 0
    non_conversion_count = 0
    original_old_word = []
    old_word = []
    old_id = []
    new_word = []
    new_id = []
    gender = []
    logit_vals = []

    sum_fem = 0
    count = 0
    for j in range(len(female_z_list)):
        # print("############", j)
        for k in range(len(female_z_list[j][0])):
            sum_fem = sum_fem + female_z_list[j][0][k]
            count+=1
    avg_sum_fem = sum_fem/count

    for j in range(len(male_z_list)):        
        for k in range(len(male_z_list[j][0])):
        # part1 = torch.tensor([female_z[0][0][15].cpu().detach().numpy() for i in female_z[0][0]])
            # new_male_z_list[j][0][0][k] = female_z_list[j][0][0][k]
            new_male_z_list[j][0][k] = avg_sum_fem
        
            
        eee = bertflow(new_male_z_list[j], reverse=True)
        eee = torch.squeeze(eee)
        ffff = decode_emb(eee)
        ffff
        for key,i in enumerate(decode_emb(m_embs[j][:,:].to(device))):
            original_old_word.append(vv[m_ids[j][key].item()])
            old_word.append(vv[i.item()])
            old_id.append(i.item())
            new_word.append(vv[ffff[key].item()])
            new_id.append(ffff[key].item())
            gender.append(1)
            logit_vals.append(m_logits[j][key].item())
    
    return original_old_word, old_word, old_id, new_word, new_id, gender, logit_vals

In [15]:
from torch.utils.data import TensorDataset
from torch.utils.data import TensorDataset

# ratio = 10000
# ratio = len(attribute_ids)


original_old_word = []
old_word = []
old_id = []
new_word = []
new_id = []
gender = []
logit_vals = []


for jj in range(1,2):
    for iteration in range(20000,21000,20000):
        print(iteration)
    
        # with open('../1_viz/attribute_embeddings/embeddings_1000.pkl', 'rb') as f:
        #     embeddings = pickle.load(f)
        # with open('../1_viz/attribute_embeddings/gender_1000.pkl', 'rb') as f:
        #     gen_labels = pickle.load(f)
        # with open('../1_viz/attribute_embeddings/token_ids_1000.pkl', 'rb') as f:
        #     attribute_ids = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/embeddings_'+str(iteration)+'.pkl', 'rb') as f:
            embeddings = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/gender_'+str(iteration)+'.pkl', 'rb') as f:
                gen_labels = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/token_ids_'+str(iteration)+'.pkl', 'rb') as f:
                attribute_ids = pickle.load(f)
        with open('../1_viz/attribute_embeddings_'+attribute_pair+'/logits_'+str(iteration)+'.pkl', 'rb') as f:
                logits = pickle.load(f)
        

        start_ind = 0
        end_ind = 0
        ratio = len(attribute_ids)
        for i in range(int(len(attribute_ids)/ratio)):
            print(i)
            end_ind+=ratio
            female_embeddings, female_attribute_ids, female_gen_labels, female_logits, male_embeddings, male_attribute_ids, male_gen_labels, male_logits = get_examples(embeddings[start_ind:end_ind], gen_labels[start_ind:end_ind], attribute_ids[start_ind:end_ind], logits[start_ind:end_ind], 1, train=False)
            female_data_pairs_test = TensorDataset(female_embeddings, female_attribute_ids, female_gen_labels, female_logits)
            male_data_pairs_test = TensorDataset(male_embeddings, male_attribute_ids, male_gen_labels, male_logits)
            
            female_dataloader = DataLoader(
                        female_data_pairs_test,  # The training samples.
                        sampler = None, #RandomSampler(female_data_pairs_test), # Select batches randomly
                        batch_size = 30 # Trains with this batch size.
                    )
            
            male_dataloader = DataLoader(
                        male_data_pairs_test,  # The training samples.
                        sampler = None, #RandomSampler(male_data_pairs_test), # Select batches randomly
                        batch_size = 30 # Trains with this batch size.
                    )
            start_ind += ratio
            
        
            try:
                avg_sum_fem = gen_flow_embs1(female_dataloader)
                original_old_word_temp, old_word_temp, old_id_temp, new_word_temp, new_id_temp, gender_temp, logit_vals_temp = gen_flow_embs(female_dataloader, male_dataloader, avg_sum_fem)
                original_old_word+= original_old_word_temp
                old_word+= old_word_temp
                old_id+= old_id_temp
                new_word+= new_word_temp
                new_id+= new_id_temp
                gender+= gender_temp
                logit_vals+= logit_vals_temp
            except:
                original_old_word_temp, old_word_temp, old_id_temp, new_word_temp, new_id_temp, gender_temp, logit_vals_temp

20000
0


/tmp/ipykernel_61203/2189383187.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  logits_list = embedding_decoder(torch.tensor(emb))


In [16]:
len(attribute_ids)
# len(female_z_list)

68733

In [17]:
lst = [original_old_word, old_word, old_id, new_word, new_id, gender, logit_vals]

In [18]:
# len(original_old_word)

In [19]:
dict = {'original_old_word': original_old_word, 'old_word': old_word, 'old_id': old_id, 'new_word': new_word, 'new_id': new_id, 'gender': gender, 'logits': logit_vals}
df = pd.DataFrame(dict)
df["count"] = 1 
df

,original_old_word,old_word,old_id,new_word,new_id,gender,logits,count
0,european,european,2647,african,3060,1,18.392763,1
1,european,european,2647,african,3060,1,20.220692,1
2,european,european,2647,african,3060,1,17.354118,1
3,europeans,europeans,13481,african,3060,1,16.877451,1
4,european,european,2647,african,3060,1,18.964178,1
...,...,...,...,...,...,...,...,...
1995,enlightenment,enlightenment,16724,malawi,18137,1,12.003662,1
1996,ee,ee,25212,ee,25212,1,12.387314,1
1997,community,community,2451,community,2451,1,12.142773,1
1998,anglo,anglo,7819,african,3060,1,12.192335,1


In [20]:
filter = df['original_old_word'].str.contains("##")
df = df[~filter]
filter = df['original_old_word'].str.contains("[CLS]")
df = df[~filter]
filter = df['original_old_word'].str.contains("[SEP]")
df = df[~filter]
filter = df['original_old_word'].str.contains("[PAD]")
df = df[~filter]

In [21]:
df.to_csv('male_female_full_pairs_'+attribute_pair+'.csv', index=False)

In [22]:
# df_stats = df
df_stats = pd.DataFrame(df.groupby(['original_old_word'])['logits'].agg('mean'))
df_stats["var"] = pd.DataFrame(df.groupby(['original_old_word'])['logits'].agg('var'))
df_stats["original_word_count"] = pd.DataFrame(df.groupby(['original_old_word'])['count'].agg('sum'))
df_stats["original_old_word"] = df_stats.index

df_stats = df_stats.reset_index(drop=True)

In [23]:
df_stats

,logits,var,original_word_count,original_old_word
0,12.675302,0.664743,6,1750
1,12.057407,NaN,1,1800s
2,12.287414,0.161626,2,18th
3,12.325362,0.087775,3,19th
4,12.371672,NaN,1,abroad
...,...,...,...,...
103,13.022544,0.849849,51,western
104,12.562697,0.148100,6,world
105,12.869451,0.369511,14,worldwide
106,12.748023,0.242753,25,xiv


In [24]:
df['original_old_word'].value_counts()

original_old_word
europe          150
european        149
europeans       140
eu              110
euro            106
               ... 
receiving         1
colonization      1
sweden            1
border            1
great             1
Name: count, Length: 108, dtype: int64

In [25]:
final_df = pd.DataFrame(df.groupby('original_old_word').new_word.agg(lambda x : x.mode())) # .reset_index()
final_df["original_old_word"] = final_df.index
final_df = final_df.reset_index(drop=True)
final_df

,new_word,original_old_word
0,the,1750
1,area,1800s
2,the,18th
3,19th,19th
4,negro,abroad
...,...,...
103,western,western
104,world,world
105,the,worldwide
106,senegal,xiv


In [26]:
df_stats = pd.merge(df_stats, final_df, how='inner', on=['original_old_word'])
df_stats

,logits,var,original_word_count,original_old_word,new_word
0,12.675302,0.664743,6,1750,the
1,12.057407,NaN,1,1800s,area
2,12.287414,0.161626,2,18th,the
3,12.325362,0.087775,3,19th,19th
4,12.371672,NaN,1,abroad,negro
...,...,...,...,...,...
103,13.022544,0.849849,51,western,western
104,12.562697,0.148100,6,world,world
105,12.869451,0.369511,14,worldwide,the
106,12.748023,0.242753,25,xiv,senegal


In [27]:
# df_stats = df_stats.dropna()

In [28]:
df_stats = df_stats.sort_values('logits', ascending=False)

In [29]:
print(df_stats.to_string())#.head(50)#[df_stats["original_old_word"]=="himself"]

        logits       var  original_word_count original_old_word                                 new_word
49   20.863379  9.032444                  149          european                                  african
48   18.078726  4.035085                  150            europe                                   africa
44   17.428793  6.373235                  110                eu                                    congo
50   16.647560  6.164968                  140         europeans                                  african
46   15.206837  4.326780                  106              euro                                    congo
69   14.981064  2.770487                    9            lisbon                                    congo
51   14.416305  2.872682                   85             euros                                      the
96   13.991384  2.257379                   94     transatlantic                                    congo
37   13.934790  2.170523                   84          

In [30]:
df_stats.to_csv('male_female_pairs_'+attribute_pair+'.csv', index=False)

In [31]:
df_stats_keep = df_stats[df_stats["original_word_count"]>2]
df_stats_discard = df_stats[df_stats["original_word_count"]<3]

In [32]:
# # df1 = df_stats[['original_old_word','new_word']]
# df_stats[df1.eq(df1.iloc[:, 0], axis=0).all(axis=1)]

In [33]:
df_stats_keep

,logits,var,original_word_count,original_old_word,new_word
49,20.863379,9.032444,149,european,african
48,18.078726,4.035085,150,europe,africa
44,17.428793,6.373235,110,eu,congo
50,16.647560,6.164968,140,europeans,african
46,15.206837,4.326780,106,euro,congo
...,...,...,...,...,...
84,12.326743,0.130239,3,ps,"[##is, malawi, tanzania]"
3,12.325362,0.087775,3,19th,19th
101,12.278841,0.019290,4,vienna,congo
81,12.257533,0.020672,4,northern,northern


In [34]:
print(df_stats_keep.to_string())

        logits       var  original_word_count original_old_word                                 new_word
49   20.863379  9.032444                  149          european                                  african
48   18.078726  4.035085                  150            europe                                   africa
44   17.428793  6.373235                  110                eu                                    congo
50   16.647560  6.164968                  140         europeans                                  african
46   15.206837  4.326780                  106              euro                                    congo
69   14.981064  2.770487                    9            lisbon                                    congo
51   14.416305  2.872682                   85             euros                                      the
96   13.991384  2.257379                   94     transatlantic                                    congo
37   13.934790  2.170523                   84          

In [35]:
print(df_stats_discard.to_string())

       logits       var  original_word_count original_old_word                  new_word
34  13.284250       NaN                    1           customs                         .
93  13.219844       NaN                    1        strasbourg                strasbourg
65  13.217318       NaN                    1             human                      ##zi
68  13.186576       NaN                    1   internationally                       the
80  12.999412       NaN                    1             north                     north
33  12.831138       NaN                    1           country                   country
59  12.824705       NaN                    1          globally                     congo
63  12.790926  0.048647                    2        hemisphere            [niger, right]
36  12.756887  0.769511                    2          domestic                    [,, .]
29  12.734962       NaN                    1         competent                      zulu
70  12.575070       N

In [36]:
for index, row in df_stats_keep.iterrows():
    try:
        if (row['original_old_word'] != row['new_word']):
            print(row['original_old_word'], row['new_word'])
    except:
        print(row['original_old_word'], row['new_word'])

european african
europe africa
eu congo
europeans african
euro congo
lisbon congo
euros the
transatlantic congo
ec african
eurasian the
europa south
member congo
continental african
anglo ['african' 'under']
nato congo
globe congo
nordic the
common central
uefa cameroon
hudson congo
reformation ['##be' '##usa' '##zee']
directive cape
worldwide the
baltic the
baroque ['congo' 'niger']
continent congo
xiv senegal
humans man
germanic african
allies ['central' 'kg' 'kong']
global ['congo' 'dr' 'global' 'nigeria']
spaniards african
1750 the
colonists ga
enlightenment ['colonial' 'malawi' 'natal']
uk ['south' 'uk']
settlers ['bam' 'bay' 'cha']
brussels congo
treaty congo
bloc congo
scandinavian zulu
spanish the
channel congo
celtic zulu
allied kg
century ['##s' '.' 'benin' 'century' 'guinea' 'sudan']
centuries ['benin' 'ivory' 'mali' 'niger' 'place']
english african
ps ['##is' 'malawi' 'tanzania']
vienna congo
encounter ##e


In [37]:
df_stats_keep

,logits,var,original_word_count,original_old_word,new_word
49,20.863379,9.032444,149,european,african
48,18.078726,4.035085,150,europe,africa
44,17.428793,6.373235,110,eu,congo
50,16.647560,6.164968,140,europeans,african
46,15.206837,4.326780,106,euro,congo
...,...,...,...,...,...
84,12.326743,0.130239,3,ps,"[##is, malawi, tanzania]"
3,12.325362,0.087775,3,19th,19th
101,12.278841,0.019290,4,vienna,congo
81,12.257533,0.020672,4,northern,northern
